In [37]:
import aquire_data
import h3
import pandas as pd
import numpy as np
import overpass
from tpot import TPOTRegressor

# Load data
# import data from VBOHCAR.xlsx to a pandas dataframe
import io
import pandas as pd
import requests
from os import listdir

# check for the vbohcar.xlsx file in the current directory
if 'VBOHCAR.xlsx' in listdir():
        # read the third sheet of the excel file
    df = pd.read_excel('VBOHCAR.xlsx', sheet_name=3)
else:
    # clone the excel file from github
    url = 'https://github.com/INFORMSJoC/2020.1022/blob/master/results/VBOHCAR.xlsx?raw=true'
    file = requests.get(url)
    file_bytes = io.BytesIO(file.content)
    # read the third sheet of the excel file
    df = pd.read_excel(file_bytes, sheet_name=3)
df.head()

,ID_OHCA,ReceivedTime,MinimumResponseTime,Latitude,Longitude,Incident_Location,X_OHCA,Y_OHCA,Z_OHCA
0,1,2017-01-01 00:49:54,5.666667,36.862471,-76.024169,1000 LINDSLEY DR_x000D_,1232.449963,-4951.984772,3826.080166
1,2,2017-01-01 06:35:29,20.750000,36.766897,-76.042337,2600 SPRINGHAVEN DR_x000D_,1232.417401,-4958.562301,3817.562796
2,3,2017-01-01 06:56:25,9.650000,36.766897,-76.042337,2600 SPRINGHAVEN DR_x000D_,1232.417401,-4958.562301,3817.562796
3,4,2017-01-01 17:05:56,11.083333,36.905880,-76.118769,4200 SHORE DR_x000D_,1223.576354,-4951.197334,3829.945177
4,5,2017-01-01 19:53:20,0.000000,36.620850,-76.090090,5200 BLACKWATER RD_x000D_,1230.619605,-4969.017372,3804.526862


In [38]:
hexid_ohca_cnt = {}
for i, row in df.iterrows():
    # get the hex_id for each row
    hex_id = h3.latlng_to_cell(np.float64(row['Latitude']), np.float64(row['Longitude']), 9)
    # if the hex_id is not in the dictionary, add it
    if hex_id not in hexid_ohca_cnt:
        hexid_ohca_cnt[hex_id] = 0
    # increment the count of OHCA in the hex_id
    hexid_ohca_cnt[hex_id] += 1

# create a dataframe from the dictionary with the hex_id as the index
ohca_df = pd.DataFrame.from_dict(hexid_ohca_cnt, orient='index', columns=['OHCA'])
print(sum(ohca_df['OHCA']))
ohca_df.head()

2706


,OHCA
892af636e43ffff,3
892af0c96abffff,2
892af63720bffff,4
892af051523ffff,2
892af6266bbffff,2


In [44]:
# load virginia beach data
# check if virginia_beach.csv is in the current directory
if 'virginia_beach.csv' in listdir():
    # read the csv file
    vb_df = pd.read_csv('virginia_beach.csv')
    # pivot the dataframe to have the hex_id as the index
    vb_df = vb_df.pivot(index='hex_id', columns=vb_df.columns)
else:
    api = overpass.API()
    vb_df = aquire_data.get_all_data('Virginia Beach', api)
vb_df.head()

ValueError: Length mismatch: Expected 986 rows, received array of length 241